In [1]:
#Import Stuff 
import os
import cv2
import math
import numpy as np

# Extract Data from Amira Files

In [2]:
# Open two empty dictionaries for later use
files = {}
landmarks = {}

for filename in os.listdir():
    if os.path.isfile(filename) and filename.endswith(".landmarkAscii") and not filename in files: 
#os.path.isfile() method in Python is used to check whether the specified path is an existing regular file or not.
#the if condition checks for a regular file and if ends with particular extension and also checks if the particular file is already in the dictionary
        with open(filename, "r") as file:
            #opens the particular file in read mode
            files[filename] = file.readlines()
            #file gets added to dictionary 'files'
            # reads all the files into a dictionary
            #readlines() reads line by line as a list

#prints all the files with given extension in the system
for filename, text in files.items(): 
    print(filename)# prints the list of files with the given extension
    #print(text)# prints the entire contents of the text file 

S051_Landmarks.landmarkAscii


## Extracting Coordinates

In [3]:
## Extracting Coordinates

# create a set of lists for coordinates
x_coordinates = [] #x-coordinate
y_coordinates = [] #y-coordinate   
coordinates = [] #x,y-coordinate put together
linecount = 0 #utility variable

#extraction of coordinates
for filename, text in files.items():
    #files is the dict; filename is teh keyword and text is teh value
    # first for loop to enter set of files
    for texts in text:
        # second for loop to enter inside inside the file and access its contents
        if texts.startswith('@1'):
            #looks for line that startswith @1 from the coordinates are listed 
             linecount = 1
             name = filename
             #print(texts)
             #print(filename)
             continue
        if linecount == 1 and texts.isspace() == False and filename==name: 
            #looks for the if @1 has been encountered and counts till an empty space
            #print(texts)
             x = list(map(float,texts.split()))[0]#convert the values into float
             y = list(map(float,texts.split()))[1]
             x_coordinates.append(x)
             y_coordinates.append(y)
             
#coordinates.extend(list(zip(x_coordinates,y_coordinates)))
print("Landmarks Acquired!")
#print(len(coordinates))

#fixing the resolution

x_coordinates = [ x/.868 for x in x_coordinates] # division to get the resolution right
y_coordinates = [ y/.868 for y in y_coordinates]

x_coordinates = [int(i) for i in x_coordinates]
y_coordinates = [int(i) for i in y_coordinates]

x = x_coordinates
y = y_coordinates

# The final Coordinates are obtained as a tuple
coordinates = list(zip(x_coordinates, y_coordinates))
#print(coordinates)
print("Number of landmarks:",len(coordinates))

Landmarks Acquired!
Number of landmarks: 177


# Access the coordinates and crop the image around the coordinates

In [4]:
#Read the image
source_path = '//ibs9010/current_data/Data_Keshav/image/MG48_3day_bs/S051_00/S51_Pia_Only.tif'
image=cv2.imread(source_path)

h = image.shape[0]
w = image.shape[1]
print("Height, Width = ",image.shape)
mini_centre_image = []
mini_partial_image = []

Height, Width =  (9536, 10340, 3)


In [5]:
def crop():
    print('Centre crop')
    for w,h in coordinates:
        #w,h = i
        #print("Null Crop:")
        #print("h:",h,"w:",w)
        #Stride_Null
        y1 = h-25
        y2 = h+25
        x1 = w-25
        x2 = w+25
        #print("The square:"+"y1:",y1,"x1:",x1,"y2:",y2,"x2:",x2)
        im1 = image[y1:y2, x1:x2]
        mini_centre_image.append(im1) 

crop()            
print("Saving",len(mini_centre_image),"single images.")

Centre crop
Saving 177 single images.


In [6]:
#Save the strided images
for k in range(len(mini_centre_image)):
    #pass
    #print("K:",k)
    Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/S051/single/'
    cv2.imwrite(os.path.join(Dest_Path,"d"+str(k)+".tif"), mini_centre_image[k])   

# Lets Stride away!


In [7]:
def right_shift_crop():
    print('right_shift_crop')

    #Top Shift
    for w,h in coordinates:
        #shift1
        #print("w:",w,"h:",h)
        #print("RightShift1")
       
        shift1 = 20
        w = w + 2 * shift1
        h = h + 2 * shift1
        #print("After right shift h:",h)
        #print("After right shift w:",w)
        y1 = h-25
        y2 = h+25
        x1 = w-25
        x2 = w+25
        #print("The square:"+"y1:",y1,"x1:",x1,"y2:",y2,"x2:",x2)
        im1 = image[y1:y2, x1:x2]
        mini_partial_image.append(im1)

right_shift_crop()            
print("Saving",len(mini_partial_image),"right shifted partial images.")

right_shift_crop
Saving 177 right shifted partial images.


In [8]:
def left_shift_crop():
    print('left_shift_crop')

    #Top Shift
    for w,h in coordinates:
        #shift1
        #print("w:",w,"h:",h)
        #print("LeftShift1")
       
        shift1 = 20
        w = w - 2 * shift1
        h = h - 2 * shift1
        #print("After left shift h:",h)
        #print("After left shift w:",w)
        y1 = h-25
        y2 = h+25
        x1 = w-25
        x2 = w+25
        #print("The square:"+"y1:",y1,"x1:",x1,"y2:",y2,"x2:",x2)
        im1 = image[y1:y2, x1:x2]
        mini_partial_image.append(im1)


left_shift_crop()            
print("Saving",len(mini_partial_image),"left shifted partial images.")

left_shift_crop
Saving 354 left shifted partial images.


In [9]:
#Save the strided images
for k in range(len(mini_partial_image)):
    #pass
    #print("K:",k)
    Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/S051/partial/'
    cv2.imwrite(os.path.join(Dest_Path,"e"+str(k)+".tif"), mini_partial_image[k])